<a href="https://colab.research.google.com/github/shuchimishra/Tensorflow_projects/blob/main/Tensorflow_Code/Timeseries/exam/Category5_daily_temperature_0.089_MAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ============================================================================================
# PROBLEM C5
#
# Build and train a neural network model using the Daily Min Temperature.csv dataset.
# Use MAE as the metrics of your neural network model.
# We provided code for normalizing the data. Please do not change the code.
# Do not use lambda layers in your model.
#
# The dataset used in this problem is downloaded from https://github.com/jbrownlee/Datasets
#
# Desired MAE < 0.19 on the normalized dataset.
# ============================================================================================

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import csv
import urllib
from tensorflow import keras

#Custom callback
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs={}):
    if logs['val_mae'] < 0.2:
      print("Stopping training since val mae is less than 0.2")
      self.model.stop_training = True

# THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# BEFORE UPLOADING YOU CAN DO IT WITH THIS

def model_forecast(model, series, window_size, batch_size):
  series = tf.expand_dims(series, axis=-1)
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size))
  ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
  forecast = model.predict(ds)
  return forecast

def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    series = tf.expand_dims(series, axis=-1)
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    ds = ds.shuffle(shuffle_buffer)
    ds = ds.map(lambda w: (w[:-1], w[1:]))
    return ds.batch(batch_size).prefetch(1)

def download_and_extract_data():
  data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
  urllib.request.urlretrieve(data_url, 'daily-min-temperatures.csv')

def get_data():
  data_file = "daily-min-temperatures.csv"
  time_step = []
  temps = []

  with open(data_file) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    next(reader)
    step = 0
    for row in reader:
      temps.append(float(row[1]))
      time_step.append(step)
      step = step + 1

  series = np.array(temps)
  time = np.array(time_step)
  return time, series


def solution_model():

  download_and_extract_data()
  time, series = get_data()

  # Normalization Function. DO NOT CHANGE THIS CODE
  min=np.min(series, axis=0)
  max=np.max(series, axis=0)
  series -= min
  series /= max

  # DO NOT CHANGE THIS CODE
  split_time=2500

  time_train = time[:split_time]
  x_train = series[:split_time]
  time_valid = time[split_time:]
  x_valid = series[split_time:]

  # DO NOT CHANGE THIS CODE
  tf.keras.backend.clear_session()
  tf.random.set_seed(42)
  np.random.seed(42)

  # DO NOT CHANGE THIS CODE
  window_size = 64
  batch_size = 256
  shuffle_buffer_size = 1000

  train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)
  valid_set=windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

  model=tf.keras.models.Sequential([
    # YOUR CODE HERE.
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='causal',input_shape=[None, 1]),
    tf.keras.layers.LSTM(64, return_sequences=True),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1)
    ])

  # Code to train and compile the model
  # Initialize learning rate
  lr = 1e-3

  # Initialize the optimizer
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr
                                     #  ,clipnorm=1
                                     ) #clipnorm to avoid nan (exploding gradient problem) # YOUR CODE HERE

  # Set the training parameters
  model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=optimizer, metrics=['mae'])

  #callback
  callback = MyCallback()
  RLP = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae",patience=3, verbose=1,mode="auto") #3
  ES = tf.keras.callbacks.EarlyStopping(monitor="val_mae",patience=5, verbose=1, mode="auto", start_from_epoch=5)

  # Train the model
  history = model.fit(train_set, epochs=500, validation_data=valid_set, verbose = 2, callbacks=[ES,RLP]) # YOUR CODE HERE

  # PASS THE NORMALIZED data IN THE FOLLOWING CODE
  rnn_forecast = model_forecast(model, series, window_size, batch_size)
  rnn_forecast = rnn_forecast[split_time - window_size:-1, 0]
  x_valid = np.squeeze(x_valid[:rnn_forecast.shape[0]])
  result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()
  print("Result is :", result) #0.090

  # YOUR CODE HERE
  return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model = solution_model()
    model.save("mymodel.h5")

Epoch 1/500
10/10 - 13s - loss: 0.2853 - mae: 0.2853 - val_loss: 0.1577 - val_mae: 0.1577 - lr: 0.0010 - 13s/epoch - 1s/step
Epoch 2/500
10/10 - 5s - loss: 0.1150 - mae: 0.1150 - val_loss: 0.0929 - val_mae: 0.0929 - lr: 0.0010 - 5s/epoch - 479ms/step
Epoch 3/500
10/10 - 6s - loss: 0.0986 - mae: 0.0986 - val_loss: 0.1034 - val_mae: 0.1034 - lr: 0.0010 - 6s/epoch - 603ms/step
Epoch 4/500
10/10 - 5s - loss: 0.0978 - mae: 0.0978 - val_loss: 0.0913 - val_mae: 0.0913 - lr: 0.0010 - 5s/epoch - 471ms/step
Epoch 5/500
10/10 - 5s - loss: 0.0936 - mae: 0.0936 - val_loss: 0.0893 - val_mae: 0.0893 - lr: 0.0010 - 5s/epoch - 549ms/step
Epoch 6/500
10/10 - 4s - loss: 0.0918 - mae: 0.0918 - val_loss: 0.0869 - val_mae: 0.0869 - lr: 0.0010 - 4s/epoch - 407ms/step
Epoch 7/500
10/10 - 6s - loss: 0.0911 - mae: 0.0911 - val_loss: 0.0862 - val_mae: 0.0862 - lr: 0.0010 - 6s/epoch - 613ms/step
Epoch 8/500
10/10 - 5s - loss: 0.0906 - mae: 0.0906 - val_loss: 0.0858 - val_mae: 0.0858 - lr: 0.0010 - 5s/epoch - 499m

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [3]:
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt
# import csv
# import urllib
# from tensorflow import keras

In [4]:
# def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
#     series = tf.expand_dims(series, axis=-1)
#     ds = tf.data.Dataset.from_tensor_slices(series)
#     ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
#     ds = ds.flat_map(lambda w: w.batch(window_size + 1))
#     ds = ds.shuffle(shuffle_buffer)
#     ds = ds.map(lambda w: (w[:-1], w[1:]))
#     return ds.batch(batch_size).prefetch(1)

In [5]:
# def download_and_extract_data():
#   data_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
#   urllib.request.urlretrieve(data_url, 'daily-min-temperatures.csv')

In [6]:
# def get_data():
#   data_file = "daily-min-temperatures.csv"
#   time_step = []
#   temps = []

#   with open(data_file) as csvfile:
#     reader = csv.reader(csvfile, delimiter=',')
#     next(reader)
#     step = 0
#     for row in reader:
#       temps.append(float(row[1]))
#       time_step.append(step)
#       step = step + 1

#   series = np.array(temps)
#   time = np.array(time_step)
#   return time, series

In [7]:
# download_and_extract_data()
# time, series = get_data()

In [8]:
# # Normalization Function. DO NOT CHANGE THIS CODE
# min=np.min(series, axis=0)
# max=np.max(series, axis=0)
# series -= min
# series /= max

In [9]:
# # DO NOT CHANGE THIS CODE
# split_time=2500

# time_train = time[:split_time]
# x_train = series[:split_time]
# time_valid = time[split_time:]
# x_valid = series[split_time:]

In [10]:
# # DO NOT CHANGE THIS CODE
# tf.keras.backend.clear_session()
# tf.random.set_seed(42)
# np.random.seed(42)

In [11]:
# # DO NOT CHANGE THIS CODE
# window_size = 64
# batch_size = 256
# shuffle_buffer_size = 1000

In [12]:
# train_set=windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

# valid_set=windowed_dataset(x_valid, window_size, batch_size, shuffle_buffer_size)

In [13]:
# for x,y in train_set.take(1):
#   print(x.shape, y.shape)
#   for i in range(1):
#     print(x[i])
#     print(y[i])

In [14]:
# model=tf.keras.models.Sequential([
#     # YOUR CODE HERE.
#     tf.keras.layers.Conv1D(filters=64, kernel_size=3, strides=1, activation='relu', padding='causal',input_shape=[None, 1]),
#     tf.keras.layers.LSTM(64, return_sequences=True),
#     tf.keras.layers.LSTM(64),
#     tf.keras.layers.Dense(100, activation='relu'),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(1)
#     ])

In [15]:
# #Custom callback
# class MyCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epochs, logs={}):
#     if logs['val_mae'] < 0.2:
#       print("Stopping training since val mae is less than 0.2")
#       self.model.stop_training = True

In [16]:
# # Code to train and compile the model
# # Initialize learning rate
# lr = 1e-3

# # Initialize the optimizer
# optimizer=tf.keras.optimizers.Adam(learning_rate=lr
#                                    #  ,clipnorm=1
#                                    ) #clipnorm to avoid nan (exploding gradient problem) # YOUR CODE HERE

# # Set the training parameters
# model.compile(loss=tf.keras.losses.mean_absolute_error, optimizer=optimizer, metrics=['mae'])

# #callback
# callback = MyCallback()
# RLP = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_mae",patience=3, verbose=1,mode="auto") #3
# ES = tf.keras.callbacks.EarlyStopping(monitor="val_mae",patience=5, verbose=1, mode="auto", start_from_epoch=5)

# # Train the model
# history = model.fit(train_set, epochs=500, validation_data=valid_set, verbose = 2, callbacks=[ES,RLP]) # YOUR CODE HERE


In [17]:

# # THIS CODE IS USED IN THE TESTER FOR FORECASTING. IF YOU WANT TO TEST YOUR MODEL
# # BEFORE UPLOADING YOU CAN DO IT WITH THIS

# def model_forecast(model, series, window_size, batch_size):
#   series = tf.expand_dims(series, axis=-1)
#   ds = tf.data.Dataset.from_tensor_slices(series)
#   ds = ds.window(window_size, shift=1, drop_remainder=True)
#   ds = ds.flat_map(lambda w: w.batch(window_size))
#   ds = ds.batch(batch_size, drop_remainder=True).prefetch(1)
#   forecast = model.predict(ds)
#   return forecast

In [18]:
# # PASS THE NORMALIZED data IN THE FOLLOWING CODE
# rnn_forecast = model_forecast(model, series, window_size, batch_size)
# rnn_forecast = rnn_forecast[split_time - window_size:-1, 0]
# x_valid = np.squeeze(x_valid[:rnn_forecast.shape[0]])
# result = tf.keras.metrics.mean_absolute_error(x_valid, rnn_forecast).numpy()
# print("Result is :", result) #0.090

In [19]:
# import matplotlib.pyplot as plt
# %matplotlib inline
# %config InlineBackend.fugure_format = 'retina'
# def plot_loss_acc(history):
#   #-----------------------------------------------------------
#   # Retrieve a list of list results on training and test data
#   # sets for each training epoch
#   #-----------------------------------------------------------
#   mae      = history.history[     'mae' ]
#   val_mae  = history.history[ 'val_mae' ]
#   loss     = history.history[    'loss' ]
#   val_loss = history.history['val_loss' ]
#   epochs   = range(len(mae)) # Get number of epochs
#   #------------------------------------------------
#   # Plot training and validation accuracy per epoch
#   #------------------------------------------------
#   plt.plot  ( epochs,     mae, label='Training mae' )
#   plt.plot  ( epochs, val_mae, label='Validation mae' )
#   plt.title ('Training and validation mae')
#   plt.grid()
#   plt.legend()
#   plt.xlabel("Epochs")
#   plt.ylabel("MAE")
#   plt.figure()
#   #------------------------------------------------
#   # Plot training and validation loss per epoch
#   #------------------------------------------------
#   plt.plot  ( epochs,     loss, label='Training loss' )
#   plt.plot  ( epochs, val_loss, label='Validation loss' )
#   plt.grid()
#   plt.legend()
#   plt.xlabel("Epochs")
#   plt.ylabel("Loss")
#   plt.title ('Training and validation loss'   )

In [20]:

# # # Plot training results
# plot_loss_acc(history)